In [11]:
import json
import pandas as pd
import os
from collections import defaultdict

# Define paths
json_folder = 'ipl_json'
output_dir = 'datasets'
os.makedirs(output_dir, exist_ok=True)

# Initialize lists to store data for existing datasets
batter_bowler_runs = []
bowler_batter_outcome = []
batter_balls = defaultdict(lambda: {'balls': 0, 'innings': 0})

# Initialize data structure for new dataset
bowler_stats = defaultdict(lambda: {'balls': 0, 'matches': 0})

# Define phase function
def get_phase(over):
    if over <= 6:
        return 'Powerplay'
    elif over <= 15:
        return 'Middleover'
    return 'Deathover'

# Process each JSON file
for filename in os.listdir(json_folder):
    if filename.endswith('.json'):
        with open(os.path.join(json_folder, filename), 'r') as f:
            match_data = json.load(f)
        
        # Extract match metadata
        match_id = filename.split('.')[0]
        people_registry = match_data['info']['registry']['people']
        players = match_data['info']['players']  # Players participating in the match
        
        # Track players (potential bowlers) who participated in this match
        match_players = set()
        for team in players:
            match_players.update(players[team])
        
        # Update match count for participating players
        for player in match_players:
            if player in people_registry:  # Ensure player has an ID
                bowler_stats[player]['matches'] += 1
        
        # Process each innings
        for inning_idx, inning in enumerate(match_data['innings'], 1):
            team = inning['team']
            batsmen_in_innings = set()  # Track batsmen who batted in this innings
            
            # Process overs
            for over_data in inning['overs']:
                over_num = over_data['over'] + 1  # 0-based to 1-based
                phase = get_phase(over_num)
                
                for delivery in over_data['deliveries']:
                    batter = delivery['batter']
                    bowler = delivery['bowler']
                    runs_scored = delivery['runs']['batter']
                    is_wicket = 'wickets' in delivery  # Check for 'wickets' key
                    
                    # Get batter and bowler IDs
                    batter_id = people_registry.get(batter, 'Unknown')
                    bowler_id = people_registry.get(bowler, 'Unknown')
                    
                    # Dataset 1: Batter-Bowler Runs (unchanged)
                    batter_bowler_runs.append({
                        'batter_name': batter,
                        'batter_id': batter_id,
                        'bowler_name': bowler,
                        'bowler_id': bowler_id,
                        'phase': phase,
                        'runs_scored': runs_scored
                    })
                    
                    # Dataset 2: Bowler-Batter Outcome (fixed)
                    outcome = 'W' if is_wicket else runs_scored
                    bowler_batter_outcome.append({
                        'bowler_name': bowler,
                        'bowler_id': bowler_id,
                        'batter_name': batter,
                        'batter_id': batter_id,
                        'phase': phase,
                        'outcome': outcome
                    })
                    
                    # Dataset 3: Batter Balls Faced (unchanged)
                    batsmen_in_innings.add(batter)
                    batter_balls[batter]['balls'] += 1
                    
                    # New Dataset: Bowler Balls Per Match
                    if bowler in match_players:  # Ensure bowler is a participant
                        bowler_stats[bowler]['balls'] += 1
            
            # Update innings count for batsmen who batted in this innings
            for batter in batsmen_in_innings:
                batter_balls[batter]['innings'] += 1

# Create DataFrames for existing datasets
batter_bowler_runs_df = pd.DataFrame(batter_bowler_runs)
bowler_batter_outcome_df = pd.DataFrame(bowler_batter_outcome)

# Dataset 3: Batter Balls Faced (unchanged)
batter_balls_data = []
for batter, stats in batter_balls.items():
    if stats['innings'] > 0:  # Only include batters who batted
        avg_balls_faced = stats['balls'] / stats['innings']
        batter_balls_data.append({
            'batter_name': batter,
            'batter_id': people_registry.get(batter, 'Unknown'),
            'average_balls_faced': avg_balls_faced
        })
batter_balls_df = pd.DataFrame(batter_balls_data)

# New Dataset: Bowler Balls Per Match
bowler_balls_data = []
for bowler, stats in bowler_stats.items():
    if stats['matches'] > 0:  # Only include bowlers who participated in matches
        avg_balls_bowled = stats['balls'] / stats['matches']
        bowler_balls_data.append({
            'bowler_name': bowler,
            'bowler_id': people_registry.get(bowler, 'Unknown'),
            'average_balls_bowled': avg_balls_bowled
        })
bowler_balls_df = pd.DataFrame(bowler_balls_data)

# Save datasets
batter_bowler_runs_df.to_csv(os.path.join(output_dir, 'batter_bowler_runs.csv'), index=False)
bowler_batter_outcome_df.to_csv(os.path.join(output_dir, 'bowler_batter_outcome.csv'), index=False)
batter_balls_df.to_csv(os.path.join(output_dir, 'batter_balls_faced.csv'), index=False)
bowler_balls_df.to_csv(os.path.join(output_dir, 'bowler_balls_per_match.csv'), index=False)

print("Datasets created: batter_bowler_runs.csv, bowler_batter_outcome.csv, batter_balls_faced.csv, bowler_balls_per_match.csv")

Datasets created: batter_bowler_runs.csv, bowler_batter_outcome.csv, batter_balls_faced.csv, bowler_balls_per_match.csv


In [21]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib
import os
from math import ceil

# Load datasets
batter_bowler_runs = pd.read_csv('datasets/batter_bowler_runs.csv')
batter_balls_faced = pd.read_csv('datasets/batter_balls_faced.csv')

# Preprocess data
# Merge datasets to ensure only batters with balls faced data are included
data = batter_bowler_runs.merge(batter_balls_faced[['batter_name', 'average_balls_faced']], 
                               on='batter_name', how='inner')

# Encode categorical features
le_batter = LabelEncoder()
le_bowler = LabelEncoder()
le_phase = LabelEncoder()

data['batter_encoded'] = le_batter.fit_transform(data['batter_name'])
data['bowler_encoded'] = le_bowler.fit_transform(data['bowler_name'])
data['phase_encoded'] = le_phase.fit_transform(data['phase'])

# Features and target
X = data[['batter_encoded', 'bowler_encoded', 'phase_encoded']]
y = data['runs_scored']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.4f}")

# Save model and encoders
joblib.dump(model, 'batting_order_model.pkl')
joblib.dump(le_batter, 'le_batter.pkl')
joblib.dump(le_bowler, 'le_bowler.pkl')
joblib.dump(le_phase, 'le_phase.pkl')

# Function to determine phase based on remaining balls
def get_phase_from_balls(remaining_balls):
    # Calculate current over (1-based)
    balls_bowled = 120 - remaining_balls  # Total balls in T20 innings = 120
    current_over = ceil(balls_bowled / 6) + 1 if balls_bowled > 0 else 1
    
    # Determine phase
    if current_over <= 6:
        return 'Powerplay'
    elif current_over <= 15:
        return 'Middleover'
    else:
        return 'Deathover'

# Function to predict next batter
def predict_next_batter(remaining_batters, current_bowler, balls_faced_df, remaining_balls):
    # Load model and encoders
    model = joblib.load('batting_order_model.pkl')
    le_batter = joblib.load('le_batter.pkl')
    le_bowler = joblib.load('le_bowler.pkl')
    le_phase = joblib.load('le_phase.pkl')
    
    # Initialize results
    batter_predictions = []
    
    # Get bowler encoding
    try:
        bowler_encoded = le_bowler.transform([current_bowler])[0]
    except ValueError:
        bowler_encoded = np.mean(le_bowler.transform(le_bowler.classes_))  # Fallback: average encoding
    
    # Predict runs for each remaining batter
    for batter in remaining_batters:
        # Get average balls faced (rounded to nearest integer)
        balls_faced = balls_faced_df[balls_faced_df['batter_name'] == batter]['average_balls_faced']
        if not balls_faced.empty:
            avg_balls = round(balls_faced.values[0])
        else:
            avg_balls = 10  # Fallback: assume 10 balls if batter not in dataset
        
        # Adjust n based on remaining balls
        if 2 * avg_balls > remaining_balls:
            n_balls = remaining_balls // 2
        else:
            n_balls = avg_balls
        
        # Get batter encoding
        try:
            batter_encoded = le_batter.transform([batter])[0]
        except ValueError:
            batter_encoded = np.mean(le_batter.transform(le_batter.classes_))  # Fallback: average encoding
        
        # Predict runs for n balls, updating phase per ball
        total_runs = 0
        balls_remaining = remaining_balls
        for i in range(n_balls):
            # Calculate phase for current ball
            current_phase = get_phase_from_balls(balls_remaining)
            
            # Get phase encoding
            try:
                phase_encoded = le_phase.transform([current_phase])[0]
            except ValueError:
                phase_encoded = le_phase.transform(['Powerplay'])[0]  # Fallback
            
            # Predict runs for this ball
            features = np.array([[batter_encoded, bowler_encoded, phase_encoded]])
            runs = model.predict(features)[0]
            total_runs += runs
            
            # Decrement remaining balls
            balls_remaining -= 1
            if balls_remaining < 0:
                break
        
        batter_predictions.append({
            'batter': batter,
            'predicted_runs': total_runs,
            'balls_faced': min(n_balls, remaining_balls)  # Ensure no negative balls
        })
    
    # Convert to DataFrame and sort by predicted runs
    predictions_df = pd.DataFrame(batter_predictions)
    predictions_df = predictions_df.sort_values(by='predicted_runs', ascending=False)
    
    return predictions_df

# Example usage
if __name__ == "__main__":
    # Example match context
    remaining_batters = ['SA Yadav', 'Tilak Varma', 'HH Pandya']
    current_bowler = 'JJ Bumrah'
    remaining_balls = 15  # Example: ~6.4 overs left, starts in Middleover
    
    # Predict next batter
    predictions = predict_next_batter(remaining_batters, current_bowler, 
                                     balls_faced_df=batter_balls_faced, 
                                     remaining_balls=remaining_balls)
    print("\nPredictions for next batter:")
    print(predictions)
    
    # Recommended batter
    best_batter = predictions.iloc[0]['batter']
    print(f"\nRecommended next batter: {best_batter}")

Mean Squared Error: 3.0053

Predictions for next batter:
        batter  predicted_runs  balls_faced
0     SA Yadav       12.099260            7
2    HH Pandya        7.423333            7
1  Tilak Varma        7.325635            7

Recommended next batter: SA Yadav


C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: U

In [57]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
import os
from math import ceil

# Load datasets
bowler_batter_outcome = pd.read_csv('datasets/bowler_batter_outcome.csv')
bowler_balls_per_match = pd.read_csv('datasets/bowler_balls_per_match.csv')

# Preprocess data
# Merge datasets to ensure only bowlers with balls per match data are included
data = bowler_batter_outcome.merge(bowler_balls_per_match[['bowler_name', 'average_balls_bowled']], 
                                  on='bowler_name', how='inner')

# Encode categorical features
le_bowler = LabelEncoder()
le_batter = LabelEncoder()
le_phase = LabelEncoder()
le_outcome = LabelEncoder()

data['bowler_encoded'] = le_bowler.fit_transform(data['bowler_name'])
data['batter_encoded'] = le_batter.fit_transform(data['batter_name'])
data['phase_encoded'] = le_phase.fit_transform(data['phase'])
data['outcome_encoded'] = le_outcome.fit_transform(data['outcome'])

# Features and target
X = data[['bowler_encoded', 'batter_encoded', 'phase_encoded']]
y = data['outcome_encoded']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Save model and encoders
joblib.dump(model, 'bowler_order_model.pkl')
joblib.dump(le_bowler, 'le_bowler.pkl')
joblib.dump(le_batter, 'le_batter.pkl')
joblib.dump(le_phase, 'le_phase.pkl')
joblib.dump(le_outcome, 'le_outcome.pkl')

# Function to determine phase based on remaining balls
def get_phase_from_balls(remaining_balls):
    balls_bowled = 120 - remaining_balls  # Total balls in T20 innings = 120
    current_over = ceil(balls_bowled / 6) + 1 if balls_bowled > 0 else 1
    if current_over <= 6:
        return 'Powerplay'
    elif current_over <= 15:
        return 'Middleover'
    else:
        return 'Deathover'

# Function to predict next bowler
def predict_next_bowler(remaining_bowlers, current_batsman, balls_bowled_dict, balls_per_match_df, remaining_balls):
    # Load model and encoders
    model = joblib.load('bowler_order_model.pkl')
    le_bowler = joblib.load('le_bowler.pkl')
    le_batter = joblib.load('le_batter.pkl')
    le_phase = joblib.load('le_phase.pkl')
    le_outcome = joblib.load('le_outcome.pkl')
    
    # Initialize results
    bowler_predictions = []
    
    # Get batsman encoding
    try:
        batter_encoded = le_batter.transform([current_batsman])[0]
    except ValueError:
        batter_encoded = np.mean(le_batter.transform(le_batter.classes_))  # Fallback: average encoding
    
    # Process each eligible bowler
    for bowler in remaining_bowlers:
        # Get max balls for this bowler
        avg_balls = balls_per_match_df[balls_per_match_df['bowler_name'] == bowler]['average_balls_bowled']
        if not avg_balls.empty:
            max_overs = round(avg_balls.values[0] / 6)
            max_balls = max_overs * 6
        else:
            max_balls = 12  # Fallback: assume 2 overs (12 balls)
        
        # Check if bowler has at least 6 balls remaining
        balls_bowled = balls_bowled_dict.get(bowler, 0)
        if max_balls - balls_bowled < 6:
            continue
        
        # Get bowler encoding
        try:
            bowler_encoded = le_bowler.transform([bowler])[0]
        except ValueError:
            bowler_encoded = np.mean(le_bowler.transform(le_bowler.classes_))  # Fallback: average encoding
        
        # Simulate one over (6 balls)
        wickets = 0
        runs = 0
        balls_remaining = remaining_balls
        for i in range(6):
            # Calculate phase for current ball
            current_phase = get_phase_from_balls(balls_remaining)
            
            # Get phase encoding
            try:
                phase_encoded = le_phase.transform([current_phase])[0]
            except ValueError:
                phase_encoded = le_phase.transform(['Powerplay'])[0]  # Fallback
            
            # Predict outcome
            features = np.array([[bowler_encoded, batter_encoded, phase_encoded]])
            outcome_encoded = model.predict(features)[0]
            outcome = le_outcome.inverse_transform([outcome_encoded])[0]
            
            # Update wickets and runs
            if outcome == 'W':
                wickets += 1
            else:
                runs += int(outcome)  # Convert outcome (0, 1, 2, 3, 4, 6) to integer
            
            # Decrement remaining balls
            balls_remaining -= 1
            if balls_remaining < 0:
                break
        
        # Store results
        bowler_predictions.append({
            'bowler': bowler,
            'wickets': wickets,
            'runs': runs,
            'average_balls_bowled': avg_balls.values[0] if not avg_balls.empty else 12
        })
    
    # Convert to DataFrame and sort
    if not bowler_predictions:
        return pd.DataFrame()  # No eligible bowlers
    predictions_df = pd.DataFrame(bowler_predictions)
    predictions_df = predictions_df.sort_values(
        by=['wickets', 'runs', 'average_balls_bowled'],
        ascending=[False, True, False]
    )
    
    return predictions_df

# Example usage
if __name__ == "__main__":
    # Example match context
    remaining_bowlers = ['JJ Bumrah', 'B Kumar', 'T Natarajan']
    current_batsman = 'SA Yadav'
    balls_bowled_dict = {'JJ Bumrah': 18, 'B Kumar': 18, 'T Natarajan': 18}  # Balls bowled so far
    remaining_balls = 12  # ~6.4 overs left, starts in Middleover
    
    # Predict next bowler
    predictions = predict_next_bowler(remaining_bowlers, current_batsman, 
                                     balls_bowled_dict, bowler_balls_per_match, 
                                     remaining_balls)
    print("\nPredictions for next bowler:")
    print(predictions)
    
    # Recommended bowler
    if not predictions.empty:
        best_bowler = predictions.iloc[0]['bowler']
        print(f"\nRecommended next bowler: {best_bowler}")
    else:
        print("\nNo eligible bowlers available.")

Accuracy: 0.3805

Predictions for next bowler:
        bowler  wickets  runs  average_balls_bowled
0    JJ Bumrah        0     0             23.947368
1      B Kumar        0    24             23.068182
2  T Natarajan        0    24             22.967213

Recommended next bowler: JJ Bumrah


C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:4

In [51]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import joblib
import os
from math import ceil
import random

# Load datasets
batter_balls_faced = pd.read_csv('datasets/batter_balls_faced.csv')
bowler_balls_per_match = pd.read_csv('datasets/bowler_balls_per_match.csv')

# Function to determine phase based on remaining balls
def get_phase_from_balls(remaining_balls):
    balls_bowled = 120 - remaining_balls
    current_over = ceil(balls_bowled / 6) + 1 if balls_bowled > 0 else 1
    if current_over <= 6:
        return 'Powerplay'
    elif current_over <= 15:
        return 'Middleover'
    else:
        return 'Deathover'

# Function to get ball outcome based on phase and bowler prediction
def get_outcome(bowling_pred, phase):
    # Base probabilities for outcomes [0, 1, 2, 3, 4, 6, W]
    base_probs = {
        'Powerplay': [0.40, 0.30, 0.08, 0.02, 0.12, 0.06, 0.02],
        'Middleover': [0.50, 0.35, 0.05, 0.01, 0.06, 0.03, 0.05],
        'Deathover': [0.35, 0.25, 0.10, 0.03, 0.15, 0.10, 0.02]
    }
    outcomes = [0, 1, 2, 3, 4, 6, 'W']
    
    if bowling_pred.empty:
        probs = np.array(base_probs[phase])
    else:
        # Sanitize predictions
        predicted_runs = max(bowling_pred.iloc[0]['runs'] / 6, 0)
        predicted_wickets = max(bowling_pred.iloc[0]['wickets'] / 6, 0)
        probs = np.array(base_probs[phase]).copy()
        
        # Adjust probabilities
        run_scaling = min(max(predicted_runs / 1.0, 0.5), 2.0)
        probs[0] /= run_scaling
        probs[1] /= run_scaling
        probs[4] *= run_scaling
        probs[5] *= run_scaling
        probs[6] = min(predicted_wickets, 0.1)
        
        # Ensure non-negative probabilities
        probs = np.clip(probs, 0, None)
        
        # Normalize probabilities
        total_prob = np.sum(probs)
        if total_prob <= 0:
            probs = np.array(base_probs[phase])
        else:
            probs = probs / total_prob
    
    # Final normalization to ensure sum == 1
    probs = np.array(probs) / np.sum(probs)
    
    # Temporary debug: Log probabilities
    print(f"Phase: {phase}, Probs: {probs.tolist()}, Sum: {np.sum(probs)}")
    
    return np.random.choice(outcomes, p=probs)

# Function to predict next batter
def predict_next_batter(remaining_batters, current_bowler, balls_faced_df, remaining_balls):
    model = joblib.load('batting_order_model.pkl')
    le_batter = joblib.load('le_batter.pkl')
    le_bowler = joblib.load('le_bowler.pkl')
    le_phase = joblib.load('le_phase.pkl')
    
    batter_predictions = []
    try:
        bowler_encoded = le_bowler.transform([current_bowler])[0]
    except ValueError:
        bowler_encoded = np.mean(le_bowler.transform(le_bowler.classes_))
    
    for batter in remaining_batters:
        balls_faced = balls_faced_df[balls_faced_df['batter_name'] == batter]['average_balls_faced']
        if not balls_faced.empty:
            avg_balls = round(balls_faced.values[0])
        else:
            avg_balls = 10
        
        if 2 * avg_balls > remaining_balls:
            n_balls = remaining_balls // 2
        else:
            n_balls = avg_balls
        
        try:
            batter_encoded = le_batter.transform([batter])[0]
        except ValueError:
            batter_encoded = np.mean(le_batter.transform(le_batter.classes_))
        
        total_runs = 0
        balls_remaining = remaining_balls
        for i in range(n_balls):
            current_phase = get_phase_from_balls(balls_remaining)
            try:
                phase_encoded = le_phase.transform([current_phase])[0]
            except ValueError:
                phase_encoded = le_phase.transform(['Powerplay'])[0]
            
            features = np.array([[batter_encoded, bowler_encoded, phase_encoded]])
            runs = model.predict(features)[0]
            total_runs += runs
            balls_remaining -= 1
            if balls_remaining < 0:
                break
        
        batter_predictions.append({
            'batter': batter,
            'predicted_runs': total_runs,
            'balls_faced': min(n_balls, remaining_balls)
        })
    
    predictions_df = pd.DataFrame(batter_predictions)
    predictions_df = predictions_df.sort_values(by='predicted_runs', ascending=False)
    return predictions_df

# Function to predict next bowler
def predict_next_bowler(remaining_bowlers, current_batsman, balls_bowled_dict, balls_per_match_df, remaining_balls):
    model = joblib.load('bowler_order_model.pkl')
    le_bowler = joblib.load('le_bowler.pkl')
    le_batter = joblib.load('le_batter.pkl')
    le_phase = joblib.load('le_phase.pkl')
    le_outcome = joblib.load('le_outcome.pkl')
    
    bowler_predictions = []
    try:
        batter_encoded = le_batter.transform([current_batsman])[0]
    except ValueError:
        batter_encoded = np.mean(le_batter.transform(le_batter.classes_))
    
    for bowler in remaining_bowlers:
        avg_balls = balls_per_match_df[balls_per_match_df['bowler_name'] == bowler]['average_balls_bowled']
        if not avg_balls.empty:
            max_overs = round(avg_balls.values[0] / 6)
            max_balls = max_overs * 6
        else:
            max_balls = 12
        
        balls_bowled = balls_bowled_dict.get(bowler, 0)
        if max_balls - balls_bowled < 6:
            continue
        
        try:
            bowler_encoded = le_bowler.transform([bowler])[0]
        except ValueError:
            bowler_encoded = np.mean(le_bowler.transform(le_bowler.classes_))
        
        wickets = 0
        runs = 0
        balls_remaining = remaining_balls
        for i in range(6):
            current_phase = get_phase_from_balls(balls_remaining)
            try:
                phase_encoded = le_phase.transform([current_phase])[0]
            except ValueError:
                phase_encoded = le_phase.transform(['Powerplay'])[0]
            
            features = np.array([[bowler_encoded, batter_encoded, phase_encoded]])
            outcome_encoded = model.predict(features)[0]
            outcome = le_outcome.inverse_transform([outcome_encoded])[0]
            
            if outcome == 'W':
                wickets += 1
            else:
                runs += int(outcome)
            
            balls_remaining -= 1
            if balls_remaining < 0:
                break
        
        bowler_predictions.append({
            'bowler': bowler,
            'wickets': wickets,
            'runs': runs,
            'average_balls_bowled': avg_balls.values[0] if not avg_balls.empty else 12
        })
    
    if not bowler_predictions:
        return pd.DataFrame()
    predictions_df = pd.DataFrame(bowler_predictions)
    predictions_df = predictions_df.sort_values(
        by=['wickets', 'runs', 'average_balls_bowled'],
        ascending=[False, True, False]
    )
    return predictions_df

# Function to get max balls for a bowler
def get_max_balls(bowler, balls_per_match_df):
    avg_balls = balls_per_match_df[balls_per_match_df['bowler_name'] == bowler]['average_balls_bowled']
    if not avg_balls.empty:
        return round(avg_balls.values[0] / 6) * 6
    return 12

# Simulator function
def simulate_match(team1_squad, team2_squad, batting_team, team1_batting_order, team2_batting_order, 
                  team1_bowling_order, team2_bowling_order):
    # Initialize data structures
    total_balls = 120
    valid_runs = [0, 1, 2, 3, 4, 6]
    
    # Validate batting team
    if batting_team not in ['Team 1', 'Team 2']:
        raise ValueError("batting_team must be 'Team 1' or 'Team 2'")
    
    # Determine batting and bowling teams
    if batting_team == 'Team 1':
        batting_team_name = 'Team 1'
        bowling_team_name = 'Team 2'
        team1_bats_first = True
    else:
        batting_team_name = 'Team 2'
        bowling_team_name = 'Team 1'
        team1_bats_first = False
    
    # Initialize simulation results
    simulations = {
        'user_defined': {
            'innings1': {'runs': 0, 'wickets': 0, 'batters': {}, 'bowlers': {}, 'commentary': []},
            'innings2': {'runs': 0, 'wickets': 0, 'batters': {}, 'bowlers': {}, 'commentary': []}
        },
        'model_predicted': {
            'innings1': {'runs': 0, 'wickets': 0, 'batters': {}, 'bowlers': {}, 'commentary': []},
            'innings2': {'runs': 0, 'wickets': 0, 'batters': {}, 'bowlers': {}, 'commentary': []}
        }
    }
    
    def initialize_innings(batting_order, bowling_team):
        batters = {b: {'runs': 0, 'balls': 0} for b in batting_order}
        bowlers = {b: {'overs': 0, 'runs': 0, 'wickets': 0} for b in bowling_team}
        return batters, bowlers
    
    def select_opening_batters(batting_team, bowling_team, remaining_balls):
        # Select a default bowler (highest average_balls_bowled) for prediction
        bowlers_df = bowler_balls_per_match[bowler_balls_per_match['bowler_name'].isin(bowling_team)]
        if not bowlers_df.empty:
            default_bowler = bowlers_df.loc[bowlers_df['average_balls_bowled'].idxmax()]['bowler_name']
        else:
            default_bowler = random.choice(bowling_team)
        
        # Predict best batters
        predictions = predict_next_batter(batting_team, default_bowler, batter_balls_faced, remaining_balls)
        if len(predictions) >= 2:
            return [predictions.iloc[0]['batter'], predictions.iloc[1]['batter']], predictions.iloc[2:]['batter'].tolist()
        elif len(predictions) == 1:
            return [predictions.iloc[0]['batter'], random.choice([b for b in batting_team if b != predictions.iloc[0]['batter']])], []
        else:
            return batting_team[:2], batting_team[2:]
    
    def simulate_innings(batting_team, bowling_team, batting_order, bowling_order, innings_num, simulation_type, team1_bats, team1_bowls, target=None):
        runs = 0
        wickets = 0
        remaining_balls = total_balls
        balls_bowled_dict = {b: 0 for b in bowling_team}
        overs_bowled_dict = {b: 0 for b in bowling_team}  # Track overs for 4-over limit
        commentary = []
        over_num = 1
        current_bowler = None
        batter_index = 0  # Track current striker
        
        # Initialize batting order
        if simulation_type == 'user_defined' or (not team1_bats):
            current_batters = [batting_order[0], batting_order[1]]
            remaining_batters = batting_order[2:]
        else:
            current_batters, remaining_batters = select_opening_batters(batting_team, bowling_team, remaining_balls)
        
        batters_stats, bowlers_stats = initialize_innings(batting_team, bowling_team)
        
        while remaining_balls > 0 and wickets < 10 and (target is None or runs <= target):
            # Start of over
            if remaining_balls % 6 == 0 or remaining_balls == total_balls:
                if simulation_type == 'user_defined' or (not team1_bowls):
                    bowler = bowling_order[over_num - 1]
                else:
                    eligible_bowlers = [b for b in bowling_team if overs_bowled_dict.get(b, 0) < 4 and b != current_bowler]
                    if over_num == 1:
                        eligible_bowlers = [b for b in bowling_team if overs_bowled_dict.get(b, 0) < 4]
                    if not eligible_bowlers:
                        # Select bowler with highest average_balls_bowled who hasn't bowled 4 overs
                        bowlers_df = bowler_balls_per_match[bowler_balls_per_match['bowler_name'].isin([b for b in bowling_team if overs_bowled_dict.get(b, 0) < 4])]
                        if not bowlers_df.empty:
                            bowler = bowlers_df.loc[bowlers_df['average_balls_bowled'].idxmax()]['bowler_name']
                        else:
                            commentary.append(f"No eligible bowlers available for Over {over_num}. Innings ends.")
                            break
                    else:
                        predictions = predict_next_bowler(eligible_bowlers, current_batters[batter_index], 
                                                         balls_bowled_dict, bowler_balls_per_match, remaining_balls)
                        bowler = predictions.iloc[0]['bowler'] if not predictions.empty else random.choice(eligible_bowlers)
                
                current_bowler = bowler
                commentary.append(f"\nOver {over_num}: {bowler} to bowl")
            
            # Simulate ball
            current_phase = get_phase_from_balls(remaining_balls)
            bowling_pred = predict_next_bowler([current_bowler], current_batters[batter_index], balls_bowled_dict, 
                                              bowler_balls_per_match, remaining_balls)
            
            outcome = get_outcome(bowling_pred, current_phase)
            runs_scored = 0 if outcome == 'W' else int(outcome)
            
            # Update stats
            batter = current_batters[batter_index]
            batters_stats[batter]['runs'] += runs_scored
            batters_stats[batter]['balls'] += 1
            bowlers_stats[current_bowler]['runs'] += runs_scored
            balls_bowled_dict[current_bowler] = balls_bowled_dict.get(current_bowler, 0) + 1
            
            if outcome == 'W':
                wickets += 1
                bowlers_stats[current_bowler]['wickets'] += 1
                if wickets < 10 and remaining_batters:
                    if simulation_type == 'user_defined' or (not team1_bats):
                        current_batters[batter_index] = remaining_batters[0]
                        remaining_batters = remaining_batters[1:]
                    else:
                        predictions = predict_next_batter(remaining_batters, current_bowler, batter_balls_faced, remaining_balls)
                        if not predictions.empty:
                            current_batters[batter_index] = predictions.iloc[0]['batter']
                            remaining_batters.remove(current_batters[batter_index])
                        else:
                            current_batters[batter_index] = remaining_batters[0]
                            remaining_batters = remaining_batters[1:]
            
            runs += runs_scored
            remaining_balls -= 1
            
            # Commentary
            commentary.append(f"Ball {over_num}.{6 - (remaining_balls % 6)}: {batter} vs {current_bowler} - {outcome}")
            
            # Rotate strike for odd runs
            if runs_scored in [1, 3]:
                batter_index = 1 - batter_index
            
            # End of over
            if remaining_balls % 6 == 0 or remaining_balls == 0 or wickets >= 10 or (target is not None and runs > target):
                overs_bowled_dict[current_bowler] = overs_bowled_dict.get(current_bowler, 0) + 1
                bowlers_stats[current_bowler]['overs'] += 1
                commentary.append(f"End of Over {over_num}: Score {runs}/{wickets}")
                commentary.append(f"Batters: {current_batters[0]} ({batters_stats[current_batters[0]]['runs']}, {batters_stats[current_batters[0]]['balls']} balls), "
                                  f"{current_batters[1]} ({batters_stats[current_batters[1]]['runs']}, {batters_stats[current_batters[1]]['balls']} balls)")
                commentary.append(f"Bowler: {current_bowler} ({bowlers_stats[current_bowler]['overs']} overs, "
                                  f"{bowlers_stats[current_bowler]['runs']} runs, {bowlers_stats[current_bowler]['wickets']} wickets)")
                over_num += 1
                batter_index = 1 - batter_index  # Switch striker
            
            if wickets >= 10 or (target is not None and runs > target):
                break
        
        return runs, wickets, batters_stats, bowlers_stats, commentary
    
    # Simulate both innings for both scenarios
    for sim_type in ['user_defined', 'model_predicted']:
        # First innings
        if team1_bats_first:
            batting_order = team1_batting_order if sim_type == 'user_defined' else team1_squad
            bowling_order = team2_bowling_order
            runs, wickets, batters_stats, bowlers_stats, commentary = simulate_innings(
                team1_squad, team2_squad, batting_order, bowling_order, 1, sim_type, team1_bats=True, team1_bowls=False
            )
            simulations[sim_type]['innings1']['runs'] = runs
            simulations[sim_type]['innings1']['wickets'] = wickets
            simulations[sim_type]['innings1']['batters'] = batters_stats
            simulations[sim_type]['innings1']['bowlers'] = bowlers_stats
            simulations[sim_type]['innings1']['commentary'] = commentary
            target = runs + 1
        else:
            batting_order = team2_batting_order
            bowling_order = team1_bowling_order if sim_type == 'user_defined' else team1_squad
            runs, wickets, batters_stats, bowlers_stats, commentary = simulate_innings(
                team2_squad, team1_squad, batting_order, bowling_order, 1, sim_type, team1_bats=False, team1_bowls=True
            )
            simulations[sim_type]['innings1']['runs'] = runs
            simulations[sim_type]['innings1']['wickets'] = wickets
            simulations[sim_type]['innings1']['batters'] = batters_stats
            simulations[sim_type]['innings1']['bowlers'] = bowlers_stats
            simulations[sim_type]['innings1']['commentary'] = commentary
            target = runs + 1
        
        # Second innings
        if team1_bats_first:
            batting_order = team2_batting_order
            bowling_order = team1_bowling_order if sim_type == 'user_defined' else team1_squad
            runs, wickets, batters_stats, bowlers_stats, commentary = simulate_innings(
                team2_squad, team1_squad, batting_order, bowling_order, 2, sim_type, team1_bats=False, team1_bowls=True, target=target
            )
            simulations[sim_type]['innings2']['runs'] = runs
            simulations[sim_type]['innings2']['wickets'] = wickets
            simulations[sim_type]['innings2']['batters'] = batters_stats
            simulations[sim_type]['innings2']['bowlers'] = bowlers_stats
            simulations[sim_type]['innings2']['commentary'] = commentary
        else:
            batting_order = team1_batting_order if sim_type == 'user_defined' else team1_squad
            bowling_order = team2_bowling_order
            runs, wickets, batters_stats, bowlers_stats, commentary = simulate_innings(
                team1_squad, team2_squad, batting_order, bowling_order, 2, sim_type, team1_bats=True, team1_bowls=False, target=target
            )
            simulations[sim_type]['innings2']['runs'] = runs
            simulations[sim_type]['innings2']['wickets'] = wickets
            simulations[sim_type]['innings2']['batters'] = batters_stats
            simulations[sim_type]['innings2']['bowlers'] = bowlers_stats
            simulations[sim_type]['innings2']['commentary'] = commentary
    
    # Display results
    for sim_type in ['user_defined', 'model_predicted']:
        print(f"\n=== {sim_type.replace('_', ' ').title()} Simulation ===")
        
        print(f"\nFirst Innings ({batting_team_name}):")
        for line in simulations[sim_type]['innings1']['commentary']:
            print(line)
        print("\nBatters:")
        for batter, stats in simulations[sim_type]['innings1']['batters'].items():
            if stats['balls'] > 0:
                print(f"{batter}: {stats['runs']} runs ({stats['balls']} balls)")
        print("\nBowlers:")
        for bowler, stats in simulations[sim_type]['innings1']['bowlers'].items():
            if stats['overs'] > 0:
                print(f"{bowler}: {stats['overs']} overs, {stats['runs']} runs, {stats['wickets']} wickets")
        
        print(f"\nSecond Innings ({bowling_team_name}):")
        for line in simulations[sim_type]['innings2']['commentary']:
            print(line)
        print("\nBatters:")
        for batter, stats in simulations[sim_type]['innings2']['batters'].items():
            if stats['balls'] > 0:
                print(f"{batter}: {stats['runs']} runs ({stats['balls']} balls)")
        print("\nBowlers:")
        for bowler, stats in simulations[sim_type]['innings2']['bowlers'].items():
            if stats['overs'] > 0:
                print(f"{bowler}: {stats['overs']} overs, {stats['runs']} runs, {stats['wickets']} wickets")

# Example usage
if __name__ == "__main__":
    # Example squads
    team1_squad = ['Q de Kock', 'KL Rahul', 'DJ Hooda', 'KH Pandya', 'A Badoni', 
                   'MP Stoinis', 'JO Holder', 'PVD Chameera', 'Avesh Khan', 'Mohsin Khan', 'Ravi Bishnoi']
    team2_squad = ['B Indrajith', 'AJ Finch', 'SS Iyer', 'N Rana', 'RK Singh', 
                   'AD Russell', 'SP Narine', 'AS Roy', 'Shivam Mavi', 'TG Southee', 'Harshit Rana']
    
    # User-defined batting and bowling orders
    team1_batting_order = ['Q de Kock', 'KL Rahul', 'DJ Hooda', 'KH Pandya', 'A Badoni', 
                           'MP Stoinis', 'JO Holder', 'PVD Chameera', 'Avesh Khan', 'Mohsin Khan', 'Ravi Bishnoi']
    team2_batting_order = ['B Indrajith', 'AJ Finch', 'SS Iyer', 'N Rana', 'RK Singh', 
                           'AD Russell', 'SP Narine', 'AS Roy', 'Shivam Mavi', 'TG Southee', 'Harshit Rana']
    team1_bowling_order = ['Mohsin Khan', 'PVD Chameera', 'Mohsin Khan', 'PVD Chameera', 'Avesh Khan', 
                           'JO Holder', 'Avesh Khan', 'Mohsin Khan', 'JO Holder', 'Ravi Bishnoi', 
                           'PVD Chameera', 'Ravi Bishnoi', 'Avesh Khan', 'Ravi Bishnoi', 'JO Holder', 
                           'Ravi Bishnoi', 'Mohsin Khan', 'JO Holder', 'Avesh Khan', 'PVD Chameera']
    team2_bowling_order = ['TG Southee', 'Shivam Mavi', 'TG Southee', 'SP Narine', 'AS Roy', 
                           'Harshit Rana', 'AS Roy', 'SP Narine', 'Shivam Mavi', 'Harshit Rana', 
                           'AS Roy', 'SP Narine', 'AD Russell', 'TG Southee', 'AD Russell', 
                           'SP Narine', 'Shivam Mavi', 'AD Russell', 'Shivam Mavi', 'TG Southee']
    
    # Simulate match
    batting_team = 'Team 1'  # Team 1 bats first
    simulate_match(team1_squad, team2_squad, batting_team, team1_batting_order, team2_batting_order, 
                   team1_bowling_order, team2_bowling_order)

C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:4

Phase: Powerplay, Probs: [0.5031446540880503, 0.3773584905660377, 0.05031446540880503, 0.012578616352201257, 0.03773584905660377, 0.018867924528301886, 0.0], Sum: 1.0


C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:4

Phase: Powerplay, Probs: [0.5031446540880503, 0.3773584905660377, 0.05031446540880503, 0.012578616352201257, 0.03773584905660377, 0.018867924528301886, 0.0], Sum: 1.0


C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:4

...

C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Phase: Deathover, Probs: [0.4810996563573883, 0.3436426116838488, 0.06872852233676977, 0.020618556701030927, 0.05154639175257732, 0.03436426116838488, 0.0], Sum: 1.0


C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Phase: Deathover, Probs: [0.4810996563573883, 0.3436426116838488, 0.06872852233676977, 0.020618556701030927, 0.05154639175257732, 0.03436426116838488, 0.0], Sum: 1.0
Phase: Deathover, Probs: [0.4810996563573883, 0.3436426116838488, 0.06872852233676977, 0.020618556701030927, 0.05154639175257732, 0.03436426116838488, 0.0], Sum: 1.0

=== User Defined Simulation ===

First Innings (Team 1):

Over 1: TG Southee to bowl
Ball 1.1: Q de Kock vs TG Southee - 0
Ball 1.2: Q de Kock vs TG Southee - 4
Ball 1.3: Q de Kock vs TG Southee - 1
Ball 1.4: KL Rahul vs TG Southee - 0
Ball 1.5: KL Rahul vs TG Southee - 0
Ball 1.6: KL Rahul vs TG Southee - 0
End of Over 1: Score 5/0
Batters: Q de Kock (5, 3 balls), KL Rahul (0, 3 balls)
Bowler: TG Southee (1 overs, 5 runs, 0 wickets)

Over 2: Shivam Mavi to bowl
Ball 2.1: Q de Kock vs Shivam Mavi - 0
Ball 2.2: Q de Kock vs Shivam Mavi - 0
Ball 2.3: Q de Kock vs Shivam Mavi - 1
Ball 2.4: KL Rahul vs Shivam Mavi - 2
Ball 2.5: KL Rahul vs Shivam Mavi - 0
Ball 2.

C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [53]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import joblib
import os
from math import ceil
import random

# Load datasets
batter_balls_faced = pd.read_csv('datasets/batter_balls_faced.csv')
bowler_balls_per_match = pd.read_csv('datasets/bowler_balls_per_match.csv')

# Function to determine phase based on remaining balls
def get_phase_from_balls(remaining_balls):
    balls_bowled = 120 - remaining_balls
    current_over = ceil(balls_bowled / 6) + 1 if balls_bowled > 0 else 1
    if current_over <= 6:
        return 'Powerplay'
    elif current_over <= 15:
        return 'Middleover'
    else:
        return 'Deathover'

# Function to get ball outcome based on phase and bowler prediction
def get_outcome(bowling_pred, phase):
    # Base probabilities for outcomes [0, 1, 2, 3, 4, 6, W]
    base_probs = {
        'Powerplay': [0.35, 0.30, 0.08, 0.02, 0.15, 0.08, 0.02],  # Increased 4s, 6s; reduced dots
        'Middleover': [0.40, 0.35, 0.05, 0.01, 0.08, 0.04, 0.07],  # Increased wickets, 4s, 6s; reduced dots
        'Deathover': [0.25, 0.20, 0.10, 0.03, 0.20, 0.15, 0.07]   # Increased 4s, 6s, wickets; reduced dots
    }
    outcomes = [0, 1, 2, 3, 4, 6, 'W']
    
    if bowling_pred.empty:
        probs = np.array(base_probs[phase])
    else:
        # Sanitize predictions
        predicted_runs = max(bowling_pred.iloc[0]['runs'] / 6, 0)
        predicted_wickets = max(bowling_pred.iloc[0]['wickets'] / 6, 0)
        probs = np.array(base_probs[phase]).copy()
        
        # Adjust probabilities
        run_scaling = min(max(predicted_runs / 1.0, 0.5), 2.5)  # Wider range for aggressive batting
        probs[0] /= run_scaling
        probs[1] /= run_scaling
        probs[4] *= run_scaling
        probs[5] *= run_scaling
        probs[6] = min(predicted_wickets * 1.5, 0.15)  # Increase wicket chance
        
        # Ensure non-negative probabilities
        probs = np.clip(probs, 0, None)
        
        # Normalize probabilities
        total_prob = np.sum(probs)
        if total_prob <= 0:
            probs = np.array(base_probs[phase])
        else:
            probs = probs / total_prob
    
    # Final normalization to ensure sum == 1
    probs = np.array(probs) / np.sum(probs)
    
    # Temporary debug: Log probabilities
    print(f"Phase: {phase}, Probs: {probs.tolist()}, Sum: {np.sum(probs)}")
    
    return np.random.choice(outcomes, p=probs)

# Function to predict next batter
def predict_next_batter(remaining_batters, current_bowler, balls_faced_df, remaining_balls):
    model = joblib.load('batting_order_model.pkl')
    le_batter = joblib.load('le_batter.pkl')
    le_bowler = joblib.load('le_bowler.pkl')
    le_phase = joblib.load('le_phase.pkl')
    
    batter_predictions = []
    try:
        bowler_encoded = le_bowler.transform([current_bowler])[0]
    except ValueError:
        bowler_encoded = np.mean(le_bowler.transform(le_bowler.classes_))
    
    for batter in remaining_batters:
        balls_faced = balls_faced_df[balls_faced_df['batter_name'] == batter]['average_balls_faced']
        if not balls_faced.empty:
            avg_balls = round(balls_faced.values[0])
        else:
            avg_balls = 10
        
        if 2 * avg_balls > remaining_balls:
            n_balls = remaining_balls // 2
        else:
            n_balls = avg_balls
        
        try:
            batter_encoded = le_batter.transform([batter])[0]
        except ValueError:
            batter_encoded = np.mean(le_batter.transform(le_batter.classes_))
        
        total_runs = 0
        balls_remaining = remaining_balls
        for i in range(n_balls):
            current_phase = get_phase_from_balls(balls_remaining)
            try:
                phase_encoded = le_phase.transform([current_phase])[0]
            except ValueError:
                phase_encoded = le_phase.transform(['Powerplay'])[0]
            
            features = np.array([[batter_encoded, bowler_encoded, phase_encoded]])
            runs = model.predict(features)[0]
            total_runs += runs
            balls_remaining -= 1
            if balls_remaining < 0:
                break
        
        batter_predictions.append({
            'batter': batter,
            'predicted_runs': total_runs,
            'balls_faced': min(n_balls, remaining_balls)
        })
    
    predictions_df = pd.DataFrame(batter_predictions)
    predictions_df = predictions_df.sort_values(by='predicted_runs', ascending=False)
    return predictions_df

# Function to predict next bowler
def predict_next_bowler(remaining_bowlers, current_batsman, balls_bowled_dict, balls_per_match_df, remaining_balls):
    model = joblib.load('bowler_order_model.pkl')
    le_bowler = joblib.load('le_bowler.pkl')
    le_batter = joblib.load('le_batter.pkl')
    le_phase = joblib.load('le_phase.pkl')
    le_outcome = joblib.load('le_outcome.pkl')
    
    bowler_predictions = []
    try:
        batter_encoded = le_batter.transform([current_batsman])[0]
    except ValueError:
        batter_encoded = np.mean(le_batter.transform(le_batter.classes_))
    
    for bowler in remaining_bowlers:
        avg_balls = balls_per_match_df[balls_per_match_df['bowler_name'] == bowler]['average_balls_bowled']
        if not avg_balls.empty:
            max_overs = round(avg_balls.values[0] / 6)
            max_balls = max_overs * 6
        else:
            max_balls = 24  # Default to 4 overs
        
        balls_bowled = balls_bowled_dict.get(bowler, 0)
        overs_bowled = balls_bowled // 6
        if overs_bowled >= 4 or max_balls - balls_bowled < 6:
            continue
        
        try:
            bowler_encoded = le_bowler.transform([bowler])[0]
        except ValueError:
            bowler_encoded = np.mean(le_bowler.transform(le_bowler.classes_))
        
        wickets = 0
        runs = 0
        balls_remaining = remaining_balls
        for i in range(6):
            current_phase = get_phase_from_balls(balls_remaining)
            try:
                phase_encoded = le_phase.transform([current_phase])[0]
            except ValueError:
                phase_encoded = le_phase.transform(['Powerplay'])[0]
            
            features = np.array([[bowler_encoded, batter_encoded, phase_encoded]])
            outcome_encoded = model.predict(features)[0]
            outcome = le_outcome.inverse_transform([outcome_encoded])[0]
            
            if outcome == 'W':
                wickets += 1
            else:
                runs += int(outcome)
            
            balls_remaining -= 1
            if balls_remaining < 0:
                break
        
        bowler_predictions.append({
            'bowler': bowler,
            'wickets': wickets,
            'runs': runs,
            'average_balls_bowled': avg_balls.values[0] if not avg_balls.empty else 12,
            'remaining_overs': (max_balls - balls_bowled) // 6
        })
    
    if not bowler_predictions:
        return pd.DataFrame()
    predictions_df = pd.DataFrame(bowler_predictions)
    predictions_df = predictions_df.sort_values(
        by=['wickets', 'remaining_overs', 'average_balls_bowled', 'runs'],
        ascending=[False, False, False, True]
    )
    return predictions_df

# Function to get max balls for a bowler
def get_max_balls(bowler, balls_per_match_df):
    avg_balls = balls_per_match_df[balls_per_match_df['bowler_name'] == bowler]['average_balls_bowled']
    if not avg_balls.empty:
        return round(avg_balls.values[0] / 6) * 6
    return 24  # Default to 4 overs

# Simulator function
def simulate_match(team1_squad, team2_squad, batting_team, team1_batting_order, team2_batting_order, 
                  team1_bowling_order, team2_bowling_order):
    # Initialize data structures
    total_balls = 120
    valid_runs = [0, 1, 2, 3, 4, 6]
    
    # Validate batting team
    if batting_team not in ['Team 1', 'Team 2']:
        raise ValueError("batting_team must be 'Team 1' or 'Team 2'")
    
    # Determine batting and bowling teams
    if batting_team == 'Team 1':
        batting_team_name = 'Team 1'
        bowling_team_name = 'Team 2'
        team1_bats_first = True
    else:
        batting_team_name = 'Team 2'
        bowling_team_name = 'Team 1'
        team1_bats_first = False
    
    # Initialize simulation results
    simulations = {
        'user_defined': {
            'innings1': {'runs': 0, 'wickets': 0, 'batters': {}, 'bowlers': {}, 'commentary': []},
            'innings2': {'runs': 0, 'wickets': 0, 'batters': {}, 'bowlers': {}, 'commentary': []}
        },
        'model_predicted': {
            'innings1': {'runs': 0, 'wickets': 0, 'batters': {}, 'bowlers': {}, 'commentary': []},
            'innings2': {'runs': 0, 'wickets': 0, 'batters': {}, 'bowlers': {}, 'commentary': []}
        }
    }
    
    def initialize_innings(batting_order, bowling_team):
        batters = {b: {'runs': 0, 'balls': 0} for b in batting_order}
        bowlers = {b: {'overs': 0, 'runs': 0, 'wickets': 0} for b in bowling_team}
        return batters, bowlers
    
    def select_opening_batters(batting_team, bowling_team, remaining_balls):
        # Select a default bowler (highest average_balls_bowled) for prediction
        bowlers_df = bowler_balls_per_match[bowler_balls_per_match['bowler_name'].isin(bowling_team)]
        if not bowlers_df.empty:
            default_bowler = bowlers_df.loc[bowlers_df['average_balls_bowled'].idxmax()]['bowler_name']
        else:
            default_bowler = random.choice(bowling_team)
        
        # Predict best batters
        predictions = predict_next_batter(batting_team, default_bowler, batter_balls_faced, remaining_balls)
        if len(predictions) >= 2:
            return [predictions.iloc[0]['batter'], predictions.iloc[1]['batter']], predictions.iloc[2:]['batter'].tolist()
        elif len(predictions) == 1:
            return [predictions.iloc[0]['batter'], random.choice([b for b in batting_team if b != predictions.iloc[0]['batter']])], []
        else:
            return batting_team[:2], batting_team[2:]
    
    def simulate_innings(batting_team, bowling_team, batting_order, bowling_order, innings_num, simulation_type, team1_bats, team1_bowls, target=None):
        runs = 0
        wickets = 0
        remaining_balls = total_balls
        balls_bowled_dict = {b: 0 for b in bowling_team}
        overs_bowled_dict = {b: 0 for b in bowling_team}
        commentary = []
        over_num = 1
        current_bowler = None
        batter_index = 0
        total_overs_bowled = 0
        
        # Initialize batting order
        if simulation_type == 'user_defined' or (not team1_bats):
            current_batters = [batting_order[0], batting_order[1]]
            remaining_batters = batting_order[2:]
        else:
            current_batters, remaining_batters = select_opening_batters(batting_team, bowling_team, remaining_balls)
        
        batters_stats, bowlers_stats = initialize_innings(batting_team, bowling_team)
        
        while remaining_balls > 0 and wickets < 10 and (target is None or runs <= target) and total_overs_bowled < 20:
            # Start of over
            if remaining_balls % 6 == 0 or remaining_balls == total_balls:
                if simulation_type == 'user_defined' or (not team1_bowls):
                    try:
                        bowler = bowling_order[over_num - 1]
                        if overs_bowled_dict.get(bowler, 0) >= 4:
                            eligible_bowlers = [b for b in bowling_team if overs_bowled_dict.get(b, 0) < 4]
                            bowler = random.choice(eligible_bowlers) if eligible_bowlers else None
                    except IndexError:
                        eligible_bowlers = [b for b in bowling_team if overs_bowled_dict.get(b, 0) < 4]
                        bowler = random.choice(eligible_bowlers) if eligible_bowlers else None
                else:
                    eligible_bowlers = [b for b in bowling_team if overs_bowled_dict.get(b, 0) < 4 and b != current_bowler]
                    if over_num == 1:
                        eligible_bowlers = [b for b in bowling_team if overs_bowled_dict.get(b, 0) < 4]
                    if not eligible_bowlers:
                        bowlers_df = bowler_balls_per_match[bowler_balls_per_match['bowler_name'].isin([b for b in bowling_team if overs_bowled_dict.get(b, 0) < 4])]
                        if not bowlers_df.empty:
                            bowler = bowlers_df.loc[bowlers_df['average_balls_bowled'].idxmax()]['bowler_name']
                        else:
                            commentary.append(f"No eligible bowlers available for Over {over_num}. Innings ends.")
                            break
                    else:
                        predictions = predict_next_bowler(eligible_bowlers, current_batters[batter_index], 
                                                         balls_bowled_dict, bowler_balls_per_match, remaining_balls)
                        bowler = predictions.iloc[0]['bowler'] if not predictions.empty else random.choice(eligible_bowlers)
                
                if bowler is None:
                    commentary.append(f"No eligible bowlers available for Over {over_num}. Innings ends.")
                    break
                
                current_bowler = bowler
                commentary.append(f"\nOver {over_num}: {bowler} to bowl")
            
            # Simulate ball
            current_phase = get_phase_from_balls(remaining_balls)
            bowling_pred = predict_next_bowler([current_bowler], current_batters[batter_index], balls_bowled_dict, 
                                              bowler_balls_per_match, remaining_balls)
            
            outcome = get_outcome(bowling_pred, current_phase)
            runs_scored = 0 if outcome == 'W' else int(outcome)
            
            # Update stats
            batter = current_batters[batter_index]
            batters_stats[batter]['runs'] += runs_scored
            batters_stats[batter]['balls'] += 1
            bowlers_stats[current_bowler]['runs'] += runs_scored
            balls_bowled_dict[current_bowler] = balls_bowled_dict.get(current_bowler, 0) + 1
            
            if outcome == 'W':
                wickets += 1
                bowlers_stats[current_bowler]['wickets'] += 1
                if wickets < 10 and remaining_batters:
                    if simulation_type == 'user_defined' or (not team1_bats):
                        current_batters[batter_index] = remaining_batters[0]
                        remaining_batters = remaining_batters[1:]
                    else:
                        predictions = predict_next_batter(remaining_batters, current_bowler, batter_balls_faced, remaining_balls)
                        if not predictions.empty:
                            current_batters[batter_index] = predictions.iloc[0]['batter']
                            remaining_batters.remove(current_batters[batter_index])
                        else:
                            current_batters[batter_index] = remaining_batters[0]
                            remaining_batters = remaining_batters[1:]
            
            runs += runs_scored
            remaining_balls -= 1
            
            # Commentary
            commentary.append(f"Ball {over_num}.{6 - (remaining_balls % 6)}: {batter} vs {current_bowler} - {outcome}")
            
            # Rotate strike for odd runs
            if runs_scored in [1, 3]:
                batter_index = 1 - batter_index
            
            # End of over
            if remaining_balls % 6 == 0 or remaining_balls == 0 or wickets >= 10 or (target is not None and runs > target):
                overs_bowled_dict[current_bowler] = overs_bowled_dict.get(current_bowler, 0) + 1
                bowlers_stats[current_bowler]['overs'] += 1
                total_overs_bowled += 1
                commentary.append(f"End of Over {over_num}: Score {runs}/{wickets}")
                commentary.append(f"Batters: {current_batters[0]} ({batters_stats[current_batters[0]]['runs']}, {batters_stats[current_batters[0]]['balls']} balls), "
                                  f"{current_batters[1]} ({batters_stats[current_batters[1]]['runs']}, {batters_stats[current_batters[1]]['balls']} balls)")
                commentary.append(f"Bowler: {current_bowler} ({bowlers_stats[current_bowler]['overs']} overs, "
                                  f"{bowlers_stats[current_bowler]['runs']} runs, {bowlers_stats[current_bowler]['wickets']} wickets)")
                over_num += 1
                batter_index = 1 - batter_index
            
            if wickets >= 10 or (target is not None and runs > target):
                break
        
        return runs, wickets, batters_stats, bowlers_stats, commentary
    
    # Simulate both innings for both scenarios
    for sim_type in ['user_defined', 'model_predicted']:
        # First innings
        if team1_bats_first:
            batting_order = team1_batting_order if sim_type == 'user_defined' else team1_squad
            bowling_order = team2_bowling_order
            runs, wickets, batters_stats, bowlers_stats, commentary = simulate_innings(
                team1_squad, team2_squad, batting_order, bowling_order, 1, sim_type, team1_bats=True, team1_bowls=False
            )
            simulations[sim_type]['innings1']['runs'] = runs
            simulations[sim_type]['innings1']['wickets'] = wickets
            simulations[sim_type]['innings1']['batters'] = batters_stats
            simulations[sim_type]['innings1']['bowlers'] = bowlers_stats
            simulations[sim_type]['innings1']['commentary'] = commentary
            target = runs + 1
        else:
            batting_order = team2_batting_order
            bowling_order = team1_bowling_order if sim_type == 'user_defined' else team1_squad
            runs, wickets, batters_stats, bowlers_stats, commentary = simulate_innings(
                team2_squad, team1_squad, batting_order, bowling_order, 1, sim_type, team1_bats=False, team1_bowls=True
            )
            simulations[sim_type]['innings1']['runs'] = runs
            simulations[sim_type]['innings1']['wickets'] = wickets
            simulations[sim_type]['innings1']['batters'] = batters_stats
            simulations[sim_type]['innings1']['bowlers'] = bowlers_stats
            simulations[sim_type]['innings1']['commentary'] = commentary
            target = runs + 1
        
        # Second innings
        if team1_bats_first:
            batting_order = team2_batting_order
            bowling_order = team1_bowling_order if sim_type == 'user_defined' else team1_squad
            runs, wickets, batters_stats, bowlers_stats, commentary = simulate_innings(
                team2_squad, team1_squad, batting_order, bowling_order, 2, sim_type, team1_bats=False, team1_bowls=True, target=target
            )
            simulations[sim_type]['innings2']['runs'] = runs
            simulations[sim_type]['innings2']['wickets'] = wickets
            simulations[sim_type]['innings2']['batters'] = batters_stats
            simulations[sim_type]['innings2']['bowlers'] = bowlers_stats
            simulations[sim_type]['innings2']['commentary'] = commentary
        else:
            batting_order = team1_batting_order if sim_type == 'user_defined' else team1_squad
            bowling_order = team2_bowling_order
            runs, wickets, batters_stats, bowlers_stats, commentary = simulate_innings(
                team1_squad, team2_squad, batting_order, bowling_order, 2, sim_type, team1_bats=True, team1_bowls=False, target=target
            )
            simulations[sim_type]['innings2']['runs'] = runs
            simulations[sim_type]['innings2']['wickets'] = wickets
            simulations[sim_type]['innings2']['batters'] = batters_stats
            simulations[sim_type]['innings2']['bowlers'] = bowlers_stats
            simulations[sim_type]['innings2']['commentary'] = commentary
    
    # Display results
    for sim_type in ['user_defined', 'model_predicted']:
        print(f"\n=== {sim_type.replace('_', ' ').title()} Simulation ===")
        
        print(f"\nFirst Innings ({batting_team_name}):")
        for line in simulations[sim_type]['innings1']['commentary']:
            print(line)
        print("\nBatters:")
        for batter, stats in simulations[sim_type]['innings1']['batters'].items():
            if stats['balls'] > 0:
                print(f"{batter}: {stats['runs']} runs ({stats['balls']} balls)")
        print("\nBowlers:")
        for bowler, stats in simulations[sim_type]['innings1']['bowlers'].items():
            if stats['overs'] > 0:
                print(f"{bowler}: {stats['overs']} overs, {stats['runs']} runs, {stats['wickets']} wickets")
        
        print(f"\nSecond Innings ({bowling_team_name}):")
        for line in simulations[sim_type]['innings2']['commentary']:
            print(line)
        print("\nBatters:")
        for batter, stats in simulations[sim_type]['innings2']['batters'].items():
            if stats['balls'] > 0:
                print(f"{batter}: {stats['runs']} runs ({stats['balls']} balls)")
        print("\nBowlers:")
        for bowler, stats in simulations[sim_type]['innings2']['bowlers'].items():
            if stats['overs'] > 0:
                print(f"{bowler}: {stats['overs']} overs, {stats['runs']} runs, {stats['wickets']} wickets")
        
        # Determine and print match result
        first_innings_runs = simulations[sim_type]['innings1']['runs']
        second_innings_runs = simulations[sim_type]['innings2']['runs']
        second_innings_wickets = simulations[sim_type]['innings2']['wickets']
        
        if second_innings_runs > first_innings_runs:
            print(f"\n{bowling_team_name} wins by {10 - second_innings_wickets} wickets!")
        elif second_innings_runs < first_innings_runs:
            print(f"\n{batting_team_name} wins by {first_innings_runs - second_innings_runs} runs!")
        else:
            print("\nThe match is a tie!")

# Example usage
if __name__ == "__main__":
    # Example squads
    team1_squad = ['Q de Kock', 'KL Rahul', 'DJ Hooda', 'KH Pandya', 'A Badoni', 
                   'MP Stoinis', 'JO Holder', 'PVD Chameera', 'Avesh Khan', 'Mohsin Khan', 'Ravi Bishnoi']
    team2_squad = ['B Indrajith', 'AJ Finch', 'SS Iyer', 'N Rana', 'RK Singh', 
                   'AD Russell', 'SP Narine', 'AS Roy', 'Shivam Mavi', 'TG Southee', 'Harshit Rana']
    
    # User-defined batting and bowling orders
    team1_batting_order = ['Q de Kock', 'KL Rahul', 'DJ Hooda', 'KH Pandya', 'A Badoni', 
                           'MP Stoinis', 'JO Holder', 'PVD Chameera', 'Avesh Khan', 'Mohsin Khan', 'Ravi Bishnoi']
    team2_batting_order = ['B Indrajith', 'AJ Finch', 'SS Iyer', 'N Rana', 'RK Singh', 
                           'AD Russell', 'SP Narine', 'AS Roy', 'Shivam Mavi', 'TG Southee', 'Harshit Rana']
    team1_bowling_order = ['Mohsin Khan', 'PVD Chameera', 'Mohsin Khan', 'PVD Chameera', 'Avesh Khan', 
                           'JO Holder', 'Avesh Khan', 'Mohsin Khan', 'JO Holder', 'Ravi Bishnoi', 
                           'PVD Chameera', 'Ravi Bishnoi', 'Avesh Khan', 'Ravi Bishnoi', 'JO Holder', 
                           'Ravi Bishnoi', 'Mohsin Khan', 'JO Holder', 'Avesh Khan', 'PVD Chameera']
    team2_bowling_order = ['TG Southee', 'Shivam Mavi', 'TG Southee', 'SP Narine', 'AS Roy', 
                           'Harshit Rana', 'AS Roy', 'SP Narine', 'Shivam Mavi', 'Harshit Rana', 
                           'AS Roy', 'SP Narine', 'AD Russell', 'TG Southee', 'AD Russell', 
                           'SP Narine', 'Shivam Mavi', 'AD Russell', 'Shivam Mavi', 'TG Southee']
    
    # Simulate match
    batting_team = 'Team 1'  # Team 1 bats first
    simulate_match(team1_squad, team2_squad, batting_team, team1_batting_order, team2_batting_order, 
                   team1_bowling_order, team2_bowling_order)

C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:4

Phase: Powerplay, Probs: [0.46204620462046203, 0.39603960396039606, 0.05280528052805281, 0.013201320132013203, 0.04950495049504951, 0.026402640264026406, 0.0], Sum: 1.0


C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:4

Phase: Powerplay, Probs: [0.46204620462046203, 0.39603960396039606, 0.05280528052805281, 0.013201320132013203, 0.04950495049504951, 0.026402640264026406, 0.0], Sum: 1.0


C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:4

...

Phase: Deathover, Probs: [0.08438818565400845, 0.06751054852320676, 0.08438818565400845, 0.025316455696202538, 0.42194092827004226, 0.31645569620253167, 0.0], Sum: 1.0


C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:4

Phase: Deathover, Probs: [0.41493775933609955, 0.33195020746887965, 0.08298755186721991, 0.024896265560165973, 0.08298755186721991, 0.06224066390041493, 0.0], Sum: 1.0
Phase: Deathover, Probs: [0.25, 0.2, 0.1, 0.03, 0.2, 0.15, 0.07], Sum: 1.0
Phase: Deathover, Probs: [0.25, 0.2, 0.1, 0.03, 0.2, 0.15, 0.07], Sum: 1.0
Phase: Deathover, Probs: [0.25, 0.2, 0.1, 0.03, 0.2, 0.15, 0.07], Sum: 1.0
Phase: Deathover, Probs: [0.25, 0.2, 0.1, 0.03, 0.2, 0.15, 0.07], Sum: 1.0
Phase: Deathover, Probs: [0.25, 0.2, 0.1, 0.03, 0.2, 0.15, 0.07], Sum: 1.0


C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\abhra\anaconda3\Lib\site-packages\sklearn\base.py:4

Phase: Deathover, Probs: [0.36900369003690037, 0.2952029520295203, 0.07380073800738007, 0.02214022140221402, 0.07380073800738007, 0.055350553505535055, 0.11070110701107011], Sum: 1.0
Phase: Deathover, Probs: [0.25, 0.2, 0.1, 0.03, 0.2, 0.15, 0.07], Sum: 1.0
Phase: Deathover, Probs: [0.25, 0.2, 0.1, 0.03, 0.2, 0.15, 0.07], Sum: 1.0
Phase: Deathover, Probs: [0.25, 0.2, 0.1, 0.03, 0.2, 0.15, 0.07], Sum: 1.0
Phase: Deathover, Probs: [0.25, 0.2, 0.1, 0.03, 0.2, 0.15, 0.07], Sum: 1.0
Phase: Deathover, Probs: [0.25, 0.2, 0.1, 0.03, 0.2, 0.15, 0.07], Sum: 1.0

=== User Defined Simulation ===

First Innings (Team 1):

Over 1: TG Southee to bowl
Ball 1.1: Q de Kock vs TG Southee - 1
Ball 1.2: KL Rahul vs TG Southee - 1
Ball 1.3: Q de Kock vs TG Southee - 1
Ball 1.4: KL Rahul vs TG Southee - 0
Ball 1.5: KL Rahul vs TG Southee - 1
Ball 1.6: Q de Kock vs TG Southee - 1
End of Over 1: Score 5/0
Batters: Q de Kock (3, 3 balls), KL Rahul (2, 3 balls)
Bowler: TG Southee (1 overs, 5 runs, 0 wickets)

Ov